In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [51]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
sample=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [52]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
import re
train=train.drop(['Ticket', 'Cabin'], axis=1)
test=test.drop(['Ticket', 'Cabin'], axis=1)
train['Name']=train['Name'].str.extract(r'(\w+[.]{1})')
test['Name']=test['Name'].str.extract(r'(\w+[.]{1})')
train['Name']=train['Name'].replace(['Don.', 'Rev.', 'Dr.', 'Mme.','Ms.', 'Major.', 'Lady.', 'Dona.',
                                     'Sir.', 'Mlle.', 'Col.', 'Capt.','Countess.', 'Jonkheer.'], 'Rare')
test['Name']=test['Name'].replace(['Don.', 'Rev.', 'Dr.', 'Mme.','Ms.', 'Major.', 'Lady.','Dona.',
                                     'Sir.', 'Mlle.', 'Col.', 'Capt.','Countess.', 'Jonkheer.'], 'Rare')

In [54]:
test['Name'].value_counts()

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Rare         7
Name: Name, dtype: int64

In [55]:
#Using QuantileTransformer to scale the continuos features
from sklearn.preprocessing import QuantileTransformer
quan_trans=QuantileTransformer(output_distribution='normal')
quan_trans.fit(train[['Age', 'Fare']])
train[['Age', 'Fare']]=quan_trans.transform(train[['Age', 'Fare']])

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [56]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,Mr.,male,-0.511517,1,0,-1.373288,S
1,2,1,1,Mrs.,female,0.658663,1,0,1.202387,C
2,3,1,3,Miss.,female,-0.166942,0,0,-0.644732,S
3,4,1,1,Mrs.,female,0.478093,1,0,0.989391,S
4,5,0,3,Mr.,male,0.478093,0,0,-0.537371,S


In [57]:
y_train=train['Survived']
X_train=train.drop(['PassengerId','Survived'], axis=1)
X_test=test.drop('PassengerId', axis=1)

### 1. Mean encodings

1.1 **Without the smooth**

We suppose that the target $Y$ is binary, that means $Y\in \{0,1\} $. The value $X_i$ of a high-cardinality categorical attribute $X$ is transformed to the conditional probability, as following:
$$X_i\rightarrow y_i=P[Y=1|X=X_i], $$
which is equivalent to

$$ y_i=P[Y=1|X=X_i]=\frac{P[Y=1, X=X_i]}{P[X=X_i]}=\frac{n_{iY}}{n_i}$$

In [58]:
def mean_enc(train, test, label, cols):
    train['label']=label
    for col in cols:
        mean=train.groupby(col)['label'].mean()
        train[col]=train[col].map(mean)
        test[col]=test[col].map(mean)
    train=train.drop('label', axis=1)
    return train, test

In [59]:
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier 

In [60]:
sgd=SGDClassifier(alpha=0.1, learning_rate='optimal', 
                  early_stopping=True, n_iter_no_change=10,
                  random_state=20)

In [61]:
#Test on the KFold
kf=KFold(shuffle=True, random_state=10)
fold=[]
tr_score=[]
val_score=[]
for k,(idx_train, idx_test) in enumerate(kf.split(X_train)):
    Xtrain, Xval=X_train.iloc[idx_train], X_train.iloc[idx_test]
    ytrain, yval=y_train.iloc[idx_train], y_train.iloc[idx_test]
    Xtrain_enc, Xval_enc=mean_enc(Xtrain, Xval, ytrain, ['Name', 'Sex', 'Embarked'])
    Xtrain_enc=Xtrain_enc.fillna(Xtrain_enc.mean())
    Xval_enc=Xval_enc.fillna(Xval_enc.mean())
    sgd.fit(Xtrain_enc, ytrain)
    tr_score.append(sgd.score(Xtrain_enc, ytrain))
    val_score.append(sgd.score(Xval_enc, yval))  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [62]:
print('Train: {}, mean: {}'.format(tr_score, np.mean(tr_score)))
print('Val: {}, mean: {}'.format(val_score, np.mean(val_score)))

Train: [0.7935393258426966, 0.782608695652174, 0.7784011220196353, 0.820476858345021, 0.7924263674614306], mean: 0.7934904738641915
Val: [0.8212290502793296, 0.7696629213483146, 0.7640449438202247, 0.7247191011235955, 0.7808988764044944], mean: 0.7721109785951918


1.2 **Mean encoder with smooth**

In order to avoid the overfitting of target encoder, we combined $2$ probabilities using the wieghting factor, as following

$$X_i\rightarrow  S_i=\lambda_i y_i+(1-\lambda_i)\bar{y},$$
where $\bar{y}$ is the mean of the target $Y$ and $\lambda_i \in [0,1]$. It deduces that
$$S_i=\lambda_i P[Y=1|X=X_i]+(1-\lambda_i) P[Y=1],$$
or
$$S_i=\lambda_i\frac{n_{iY}}{n_i}+(1-\lambda_i)\frac{n_Y}{n}.$$
Normally, we take $\lambda_i=\frac{n_i}{n_i+m}$, where $m$ is a parameter, depends on data. In this case, 
$$ S_i=\frac{n_{iY}+m\bar{y}}{n_i+m}.$$

In [63]:
#Mean encoding with smooth, take m=300.
def mean_enc_smooth(train, test, label, cols):
    train['label']=label
    mean=np.mean(label)
    for col in cols:
        smooth=train.groupby(col)['label'].agg(['sum','count'])
        smooth['enc']=(smooth['sum']+300*mean)/(smooth['count']+300)
        train[col]=train[col].map(smooth['enc'])
        test[col]=test[col].map(smooth['enc'])
    train=train.drop('label', axis=1)
    return train, test


In [64]:
#Test on the KFold
kf=KFold(shuffle=True, random_state=10)
fold=[]
tr_score=[]
val_score=[]
for k,(idx_train, idx_test) in enumerate(kf.split(X_train)):
    Xtrain, Xval=X_train.iloc[idx_train], X_train.iloc[idx_test]
    ytrain, yval=y_train.iloc[idx_train], y_train.iloc[idx_test]
    Xtrain_enc, Xval_enc=mean_enc_smooth(Xtrain, Xval, ytrain, ['Name', 'Sex', 'Embarked'])
    Xtrain_enc=Xtrain_enc.fillna(Xtrain_enc.mean())
    Xval_enc=Xval_enc.fillna(Xval_enc.mean())
    sgd.fit(Xtrain_enc, ytrain)
    tr_score.append(sgd.score(Xtrain_enc, ytrain))
    val_score.append(sgd.score(Xval_enc, yval))  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [65]:
print('Train: {}, mean: {}'.format(tr_score, np.mean(tr_score)))
print('Val: {}, mean: {}'.format(val_score, np.mean(val_score)))

Train: [0.7148876404494382, 0.726507713884993, 0.7405329593267882, 0.7335203366058906, 0.729312762973352], mean: 0.7289522826480924
Val: [0.7653631284916201, 0.702247191011236, 0.7078651685393258, 0.7303370786516854, 0.7134831460674157], mean: 0.7238591425522567


1.3 **Leave one out**

In order to reduce the variance, when caculating the mean target, we leave out the current row's target. Normally, we apply this method only on the training set.



In [66]:
#Mean encoder with Leave-One-Out.
def mean_enc_loo(train, label, cols):
    train['label']=label
    for col in cols:
        train_sum = all_data.groupby(col)['label'].transform('sum')
        train_n_objects = all_data.groupby(col)['label'].transform('count')
        train[col] = (train_sum - train['label']) / (train_n_objects - 1)
    train=train.drop('label', axis=1)
    return train


### 2. Weight of Evidence (WOE) and Information Value (IV)
2.1 **The Weight of Evidence (woe)** 
The woe is defined by
$$woe_i=ln \frac{P(X=x_i|Y=1)}{P(X=x_i|Y=0)},$$
where 
$$P(X=x_i|Y=1)=\frac{n_{iY}}{n_{Y}} \text{ and } P(X=x_i|Y=0)=\frac{n_i-n_{iY}}{n-n_Y}$$
In order to avoid the division by $0$ and the logarit of $O$, we replace $P(X=x_i|Y=1)$ by 
$\frac{n_{iY}+0.5}{n_{Y}} $ and $P(X=x_i|Y=0)$ by $\frac{n_i-n_{iY}+0.5}{n-n_Y}$

2.2 **The Information Value**
IV is determied by
$$IV_i=\left[P(X=x_i|Y=1)-P(X=x_i|Y=0)\right].woe_i$$


In [67]:
def woe_iv(train, test, label, cols, method=None):
    train['label']=label
    for col in cols:
        gr=train.groupby(col)['label'].agg(['sum','count'])
        gr['non_survived']=gr['count']-gr['sum']
        gr=gr.drop('count', axis=1)
        total=gr.sum(axis=0)
        gr['p_survived']=gr['sum'] / total[0]
        gr['p_non_survived']=gr['non_survived'] / total[1]
        gr['woe']=np.log(gr['p_survived'] / gr['p_non_survived'])
        gr['iv']=(gr['p_survived']-gr['p_non_survived'])*gr['woe']
        if method==None:
            train[col]=train[col].map(gr['woe'])
            test[col]=test[col].map(gr['woe'])
        else:
            train[col]=train[col].map(gr['iv'])
            test[col]=test[col].map(gr['iv'])
    train=train.drop('label', axis=1)
    return train, test

In [68]:
#Test on the KFold
kf=KFold(shuffle=True, random_state=10)
fold=[]
tr_score=[]
val_score=[]
for k,(idx_train, idx_test) in enumerate(kf.split(X_train)):
    Xtrain, Xval=X_train.iloc[idx_train], X_train.iloc[idx_test]
    ytrain, yval=y_train.iloc[idx_train], y_train.iloc[idx_test]
    Xtrain_woe, Xval_woe=woe_iv(Xtrain, Xval, ytrain, ['Name', 'Sex', 'Embarked'])
    Xtrain_woe.fillna(Xtrain_woe.mean(), inplace=True)
    Xval_woe.fillna(Xval_woe.mean(), inplace=True)
    sgd.fit(Xtrain_woe, ytrain)
    tr_score.append(sgd.score(Xtrain_woe, ytrain))
    val_score.append(sgd.score(Xval_woe, yval))  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [69]:
print('Train: {}, mean: {}'.format(tr_score, np.mean(tr_score)))
print('Val: {}, mean: {}'.format(val_score, np.mean(val_score)))

Train: [0.7879213483146067, 0.8022440392706872, 0.8092566619915849, 0.8106591865357644, 0.7924263674614306], mean: 0.8005015207148147
Val: [0.8156424581005587, 0.7471910112359551, 0.8258426966292135, 0.7415730337078652, 0.8258426966292135], mean: 0.7912183792605612
